In [1]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Function to extract features from audio
def extract_features(audio_file):
    y, sr = librosa.load(audio_file, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

# Load dataset and extract features (replace with your dataset)
def load_dataset():
    scream_files = ["Scream+21.wav", "Scream+27.wav", "Scream+28.wav", "Scream+29.wav", "Screams+1.wav", "Screams+2.wav"]  
    non_scream_files = ["talk1.wav", "talk2.wav"]  

    X, y = [], []
    for file in scream_files:
        X.append(extract_features(file))
        y.append(1)  # Label for scream
    for file in non_scream_files:
        X.append(extract_features(file))
        y.append(0)  # Label for non-scream

    return np.array(X), np.array(y)

# Train and evaluate model
X, y = load_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = SVC(kernel="linear", probability=True)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Prediction on new data
def predict_scream(audio_file):
    features = extract_features(audio_file).reshape(1, -1)
    prediction = model.predict(features)
    return "SOS Triggered" if prediction[0] == 1 else "No distress detected"

# Example usage
print(predict_scream("Scream+21.wav"))


Model Accuracy: 100.00%
SOS Triggered


In [4]:
pip install SpeechRecognition

   ---------------------------------------- 0.0/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/32.8 MB 3.1 MB/s eta 0:00:11
   ---------------------------------------- 0.2/32.8 MB 4.4 MB/s eta 0:00:08
    --------------------------------------- 0.5/32.8 MB 3.8 MB/s eta 0:00:09
    --------------------------------------- 0.8/32.8 MB 4.8 MB/s eta 0:00:07
   - -------------------------------------- 1.2/32.8 MB 5.8 MB/s eta 0:00:06
   - -------------------------------------- 1.6/32.8 MB 6.7 MB/s eta 0:00:05
   -- ------------------------------------- 2.0/32.8 MB 6.8 MB/s eta 0:00:05
   --- ------------------------------------ 2.6/32.8 MB 7.8 MB/s eta 0:00:04
   --- ------------------------------------ 3.1/32.8 MB 8.1 MB/s eta 0:00:04
   ---- ----------------------------------- 3.5/32.8 MB 8.2 MB/s eta 0:00:04
   ---- ----------------------------------- 3.9/32.8 MB 8.2 MB/s eta 0:00:04
   ----- ---------------------------------- 4.4/32.8 MB 8.8 MB/s eta 0:00:04
   ---

In [6]:
import speech_recognition as sr
from collections import Counter
import matplotlib.pyplot as plt

# Function to detect keywords in audio
def detect_keywords(audio_file, keywords=["call", "help"]):
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio_file) as source:
            audio = recognizer.record(source)
        transcript = recognizer.recognize_google(audio).lower()
        detected_keywords = [word for word in transcript.split() if word in keywords]
        return "SOS Triggered" if detected_keywords else "No distress detected", transcript, detected_keywords
    except Exception as e:
        return f"Error processing audio: {e}", None, []

# Evaluate keyword detection
def evaluate_model(audio_files, labels, keywords=["call", "help"]):
    results = []
    for i, file in enumerate(audio_files):
        result, transcript, detected_keywords = detect_keywords(file, keywords)
        results.append(1 if "SOS Triggered" in result else 0)
        print(f"File: {file} | Label: {labels[i]} | Result: {result} | Keywords: {detected_keywords}")

    accuracy = np.mean([1 if results[i] == labels[i] else 0 for i in range(len(labels))])
    print(f"Keyword Detection Accuracy: {accuracy * 100:.2f}%")
    
    
# Example Usage
audio_files = ["Call1.wav", "call_help.wav"]  # Replace with your audio files
labels = [1, 0]  # 1 for distress, 0 for normal
evaluate_model(audio_files, labels)


File: Call1.wav | Label: 1 | Result: SOS Triggered | Keywords: ['call', 'call', 'call', 'call', 'call', 'call']
File: call_help.wav | Label: 0 | Result: SOS Triggered | Keywords: ['help', 'call', 'help', 'call', 'help', 'help', 'call', 'help']
Keyword Detection Accuracy: 50.00%
